## Coordinate Transformations

In [6]:
# global constants
g_maxAngVel = pi / 2 # move in azimuth or elevation at max rate of 90 deg / sec
g_maxScanRate = 1000 # scans points at the rate of 1000 pts/sec

# function to convert points from cartesian Part Coordiante System to cartesian Machine Coordinate System 
# input:  mcs: 4x4 transformation matrix
#         pts: nx3 list of points in PCS
# output: nx3 list of points in MCS
function pcs_to_mcs_xyz(mcs, pts)
    # transform all points into LR Cartesian coordinates
    # homogenize the pts with a 1
    qty = size(pts,1)
    tmp = [pts ones(qty,1)]
    # multiply by inverse of the LR's location/orientation
    ret = inv(mcs) * tmp'
    return ret[1:3,:]'
end;

# function to convert points from cartesian Part Coordiante System to cartesian Machine Coordinate System 
# input:  mcs: 4x4 transformation matrix
#         pts: nx3 list of points in MCS
# output: nx3 list of points in PCS
function mcs_to_pcs_xyz(mcs, pts)
    # transform all points into LR Cartesian coordinates
    # homogenize the pts with a 1
    qty = size(pts,1)
    tmp = [pts ones(qty,1)]
    # multiply by inverse of the LR's location/orientation
    ret = mcs * tmp'
    return ret[1:3,:]'
end;

# function to convert points from cartesian to spherical
# input:  pts: nx3 list of xyz (cartesian) points
# output: nx3 list of spherical points (range, theta, phi)
function xyz_to_sph(pts)
    r = [vecnorm(pts[i,:]) for i=1:size(pts,1)]
    r2= [vecnorm(pts[i,1:2]) for i=1:size(pts,1)]
    t = atan2.(r2, pts[:,3])
    p = atan2.(pts[:,2], pts[:,1])
    return [r t p]
end;

# function to convert points from spherical to cartesian
# input:  pts: nx3 list of (r,t,p) (spherical) points
# output: nx3 list of cartesian points (xyz)
function sph_to_xyz(pts)
    # multiply by inverse of the LR's location/orientation
    x = pts[:,1] .* sin.(pts[:,2]) .* cos.(pts[:,3]) # rsinθcosϕ
    y = pts[:,1] .* sin.(pts[:,2]) .* sin.(pts[:,3]) # rsinθsinϕ
    z = pts[:,1] .* cos.(pts[:,2])                   # rcosθ
    return [x y z]
end;


# function to simulate a scan from one spherical coord to another
# input: from: spherical coordinate triple (radius, theta, phi) to start from
#        to:   spherical coordinate triple (radius, theta, phi) to end at
# output: n x 3 list of spherical points
function scan_from_to(from, to)
    delta = to .- from
    time = maximum(abs.(delta[2:3])) / g_maxAngVel
    qty = floor(time * g_maxScanRate)
    rgs = linspace.(from, to, maximum([qty,5])) # want at least 5 points.
    return [collect(rgs[1]) collect(rgs[2]) collect(rgs[3])]
end

# function to simulate a scan through a list of (spherical) points
# input:  list of points (spherical) in the order to be visited
# output: list of points (spherical) of the simulated scan points
function scan_tour(pts)
    ret = []
    for i = 1:size(pts,1)-1
        line = scan_from_to(pts[i,:], pts[i+1,:])
        ret = [ret; line]
    end
    return ret
end

function test_transforms()
    # load the feature file
    data = readcsv("50_Holes_FrontLeft.csv");
    pts_pcs = convert(Array{Float64, 2}, data[:,2:4]) # feature locations in PCS
    lr_tfm = readcsv("Location1.csv") # transformation matrix for LR's position

    # convert to spherical
    pts_mcs = pcs_to_mcs_xyz(lr_tfm, pts_pcs) # convert feature locations to MCS
    pts_lr = xyz_to_sph(pts_mcs) # convert feature locations to spherical

    # round trip as a test
    pts_mcs_back = sph_to_xyz(pts_lr)
    pts_pcs_back = mcs_to_pcs_xyz(lr_tfm, pts_mcs_back)

    # points should be the same
    diff = pts_pcs - pts_pcs_back
    display(maximum(diff))
end

function test_scan()
    # load data.
    data = readcsv("50_Holes_FrontLeft.csv");
    pts_pcs = convert(Array{Float64, 2}, data[:,2:4]) # feature locations in PCS
    lr_tfm = readcsv("Location1.csv") # transformation matrix for LR's position

    # shuffle an index vector
    idx = Vector(1:size(pts_pcs,1))
    rng = MersenneTwister(1234)
    idx = shuffle(rng, idx)
    
    # shuffle input pts and convert them to spherical
    sph = xyz_to_sph(pcs_to_mcs_xyz(lr_tfm, pts_pcs[idx,:]))

    # generate scan data
    scan = scan_tour(sph)
    
    # back to pcs
    scan = mcs_to_pcs_xyz(lr_tfm, sph_to_xyz(scan))
    println(size(scan))
    
    # save
    writecsv("scan1.csv", scan)
end

test_transforms()
test_scan()

8.881784197001252e-16

(9968, 3)


## TSP

In [2]:
using JuMP, Gurobi

function find_subtour(edges, n)
    visited = [false for i=1:n]
    cycles=[]
    lengths=[]
    selected=[[] for i=1:n]
    for e in edges
        push!(selected[e[1]], e[2])
    end
    while true
        current = find(visited .== false)[1]
        thiscycle = [current]
        while true
            visited[current] = true
            neighbors = []
            for x in selected[current]
                if visited[x] == false
                    push!(neighbors, x)
                end
            end
            if length(neighbors) == 0
                break
            end
            current = neighbors[1]
            push!(thiscycle, current)
        end
        push!(cycles, thiscycle)
        push!(lengths, length(thiscycle))
        if sum(lengths) == n
            break
        end
    end
    #display(cycles)
    return cycles[indmin(lengths)]
end

function remove_dummy_node(tour)
    len = length(tour)
    idx_dummy = indmax(tour)
    return [tour[(i+idx_dummy)%len+1] for i=0:len-2]
end

function tsp(cost, return_to_start)
    n = size(cost,1)
    if !return_to_start
        # if we don't need to form a loop, add a dummy node of cost 0 to everywhere.
        # our non-cyclic path will start from the node following this node
        # and end at the node preceding this node.
        cost = [cost zeros(size(cost,1),1)]
        cost = [cost ; zeros(1, size(cost,2))]
        n += 1
    end

    # TSP solve
    m = Model(solver=GurobiSolver(OutputFlag=0))    
    # use the symmetric matrix formulation & subtour elimination described at
    #  http://examples.gurobi.com/traveling-salesman-problem/
    @variable(m, z[i=1:n, j=1:n], Bin, Symmetric)
    @objective(m, Min, 0.5*sum(z.*cost)) # minimize the sum of movement times
    @constraint(m, [i in 1:n], sum(z[i,j] for j=1:n) == 2 ) # one in and one out edge
    @constraint(m, [ii in 1:n], z[ii,ii] == 0) # can't transition to self

    num_lazy_constraints = 0
    function subtour_callback(cb)
        soln = getvalue(z)
        # check for subtours
        selected = [[i,j] for i=1:n, j=1:n if soln[i,j] > 0.5]
        subtour = find_subtour(selected, n)
        if length(subtour) == n
            return
        end
        # add constraint to prohibit these subtours
        len = length(subtour)
        subtour = sort(subtour)
        num_lazy_constraints += 1
        @lazyconstraint(cb, sum( z[subtour[j],subtour[i]] for i = 1:len, j=i:len ) <= len-1)
    end

    # add the lazy constraint callback and solve
    addlazycallback(m, subtour_callback)
    solve(m)
    soln = getvalue(z)
                
    # extract feature order
    selected = [[i,j] for i=1:n, j=1:n if soln[i,j] > 0.5]
    tour = find_subtour(selected, n)
    if !return_to_start
        tour = remove_dummy_node(tour)
    end
    return [tour, getobjectivevalue(m)]
end;

## Driver

In [3]:
# function that computes the travel time from spherical coordinate 'from' to 'to'
# (assumes instantaneous acceleration to max velocity)
travel_time(from, to) = maximum(abs.((to .- from)[2:3])) / g_maxAngVel

# function that takes a list of nx3 spherical coordinates and returns an n x n 
# matrix where A(i,j) where  the time it takes to move from point i to point j
cost_matrix(sph) = [travel_time(sph[i,:],sph[j,:]) for i=1:size(sph,1), j=1:size(sph,1)]

# finds minimal path to scan features given an LR location, and
# computes (simulated) scan data along that path.
# input:  features: matrix of features in feature file format.
#         lr_location: 4x4 transformation matrix of LR's position in PCS.
#         optimize: boolean indicating whether to optimize the path or not.
# output: list containing:
#           list of indices of order that the features were measured
#           time it took to travel to the entire path
#           elapsed (computational) time to optimize the path
#           simulated scan points along the path
function scan_feature_path(features, lr_location, optimize)
    # xyz location of features
    pts_pcs = convert(Array{Float64, 2}, features[:,2:4]) # feature locations in PCS

    # spherical coordinates with origin at laser radar
    sph = xyz_to_sph(pcs_to_mcs_xyz(lr_location, pts_pcs))
    cost = cost_matrix(sph) # cost to travel from feature i to feature j
    
    if optimize
        # optimize
        tm = @timed(tsp(cost, false))
    else
        # don't optimize. scan in current order
        n = size(cost,1)
        tm = @timed([[i for i=1:n], sum(cost[i,i+1] for i=1:n-1)])
    end
    tour = tm[1]
    push!(tour, tm[2])
    
    # simulate a scan of the features
    scan = scan_tour(sph[tour[1],:])
    # back to pcs
    scan = mcs_to_pcs_xyz(lr_location, sph_to_xyz(scan))
    push!(tour, scan)
    return tour
end

# test harness
function test(featureFile, locationFile, outScan_no_opt, outScan_opt)            
    features = readcsv(featureFile); # feature file
    lr_position = readcsv(locationFile) # transformation matrix for LR's positionend

    scan_no_opt = scan_feature_path(features, lr_position, false)
    scan_opt = scan_feature_path(features, lr_position, true)
    
    @printf("Feature path without optimization: %.6f sec\n", scan_no_opt[2])
    @printf("Feature path with optimization:    %.6f sec\n", scan_opt[2])
    @printf("Time savings:                      %.6f sec\n", scan_no_opt[2] - scan_opt[2])
    @printf("Optimal feature ordering:\n")
    println(scan_opt[1])
    @printf("Computational time to optimize:    %.6f sec\n", scan_opt[3])
    @printf("Writing %d non-optimized scan points to '%s'\n", size(scan_no_opt[4],1), outScan_no_opt)
    @printf("Writing %d optimized scan points to '%s'\n", size(scan_opt[4],1), outScan_opt)
    writecsv(outScan_no_opt, scan_no_opt[4])
    writecsv(outScan_opt, scan_opt[4])
end
        
test("50_Holes_FrontLeft.csv", "Location1.csv", "50_left_front_scan_no_opt.csv", "50_left_front_scan_opt.csv")

Academic license - for non-commercial use only
Feature path without optimization: 11.792777 sec
Feature path with optimization:    1.561622 sec
Time savings:                      10.231154 sec
Optimal feature ordering:
[29, 14, 35, 39, 24, 41, 20, 12, 21, 49, 47, 18, 23, 36, 28, 32, 45, 37, 1, 10, 46, 33, 44, 25, 27, 15, 5, 34, 50, 48, 3, 19, 40, 13, 4, 43, 7, 22, 8, 17, 38, 6, 2, 31, 30, 26, 9, 42, 16, 11]
Computational time to optimize:    4.079195 sec
Writing 11771 non-optimized scan points to '50_left_front_scan_no_opt.csv'
Writing 1541 optimized scan points to '50_left_front_scan_opt.csv'


## Test Different LR Locations on same Feature File

In [8]:
# location of LR is 0,4,0
test("50_Holes_FrontLeft.csv", "LocationFront.csv", "ignore.csv", "50feature_LRFront.csv")
# location of LR is 3,1,0
test("50_Holes_FrontLeft.csv", "LocationLeft.csv", "ignore.csv", "50feature_LRLeft.csv")

Academic license - for non-commercial use only
Feature path without optimization: 8.281533 sec
Feature path with optimization:    1.213455 sec
Time savings:                      7.068077 sec
Optimal feature ordering:
[11, 16, 2, 6, 42, 9, 26, 30, 31, 38, 8, 17, 43, 4, 22, 7, 13, 40, 19, 50, 5, 15, 27, 44, 25, 28, 32, 45, 36, 23, 18, 47, 49, 37, 1, 10, 46, 33, 34, 48, 3, 24, 41, 20, 12, 21, 39, 35, 14, 29]
Computational time to optimize:    0.187134 sec
Writing 8259 non-optimized scan points to 'ignore.csv'
Writing 1204 optimized scan points to '50feature_LRFront.csv'
Academic license - for non-commercial use only
Feature path without optimization: 56.738953 sec
Feature path with optimization:    4.856255 sec
Time savings:                      51.882698 sec
Optimal feature ordering:
[11, 16, 26, 9, 42, 31, 30, 38, 17, 43, 4, 5, 13, 15, 40, 50, 19, 27, 44, 25, 45, 32, 28, 36, 23, 48, 3, 18, 47, 49, 21, 12, 20, 41, 24, 14, 29, 35, 39, 37, 1, 10, 46, 33, 34, 7, 22, 8, 6, 2]
Computational t